In [1]:
!pip install pandas
!pip install konlpy


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
'''
Cleaning_V1
'…', '...', '·' 제거 후 strip() 적용
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text(text):
    # Replace '… ', '...', and '·' with ' ' and then apply strip()
    cleaned_text = text.replace('…', ' ').replace('...', ' ').replace('·', ' ').strip()
    return cleaned_text

df['text'] = df['text'].apply(clean_text)

output_file_path = '../data/train_cleaning_v1.csv'
df.to_csv(output_file_path, index=False)

In [3]:
import pandas as pd
import re

'''
Cleaning_V3
Replace all digits with '0'
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text_v3(text):
    # 모든 숫자를 '0'으로 치환
    cleaned_text = re.sub(r'\d', '0', text)
    return cleaned_text

df['text'] = df['text'].apply(clean_text_v3)

output_file_path = '../data/train_cleaning_v3.csv'
df.to_csv(output_file_path, index=False)

In [31]:
from konlpy.tag import Okt
def extract_nouns(sentence):
    okt = Okt()
    nouns = okt.nouns(sentence)
    return nouns

# Example usage
korean_sentence = "SOil 1조2천억원 경유·납사·항공유 공급계약"
result_nouns = extract_nouns(korean_sentence)

print("Original sentence:", korean_sentence)
print("Extracted nouns:", result_nouns)

Original sentence: SOil 1조2천억원 경유·납사·항공유 공급계약
Extracted nouns: ['경유', '납사', '항', '공유', '공급', '계약']


In [2]:
# Install MeCab and its dependencies
!apt-get install mecab libmecab-dev mecab-ipadic-utf8

# Install the MeCab Python wrapper
!pip install mecab-python3

!apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

!apt-get install mecab-ko-dic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.24).
git is already the newest version (1:2.17.1-1ubuntu0.18).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
mecab-ko is already installed
Install mecab-ko-dic
Install mecab-ko-dic
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:-

In [8]:
!echo `mecab-config --dicdir`"/mecab-ko-dic"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic


In [1]:
from konlpy.tag import Mecab
import pandas as pd
import re

'''
Cleaning_V4
명사만 남기고 모든 조사, 동사 등을 제거함
기존에 붙어 있던 명사는 붙이고, 떨어져 있던 명사는 떨어트림
'''
def extract_nouns(sentence):
    mecab = Mecab('/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic')
    tokens = mecab.pos(sentence)

    # Filter out unwanted tokens (keeping nouns, numbers, and English words)
    filtered_tokens = [word for word, pos in tokens if pos.startswith('N') or pos.startswith('V') or pos.startswith('M') or pos == 'SN' or pos == 'SL']
    result_tokens = []
    
    current_token = ''    
    next_token = ''
    for i in range(len(filtered_tokens) - 1):
        current_token = filtered_tokens[i] if current_token == '' else current_token
        next_token = filtered_tokens[i + 1]
        
        pattern = re.escape(current_token + next_token)
        if re.search(pattern, sentence):
            current_token += next_token
        else:
            result_tokens.append(current_token)
            current_token = ''
            
    if current_token:
        result_tokens.append(current_token)
    else:
        result_tokens.append(next_token)
        
    if len(filtered_tokens) == 1:
        return filtered_tokens
    
    return result_tokens

file_path = "../data/train_cleaning_v3.csv"
output_file_path = "../data/train_cleaning_v6.csv"
df = pd.read_csv(file_path)
# Create a temporary DataFrame with the same structure as df
temp_df = df.copy()

# Apply extract_nouns to all rows of temp_df['text']
temp_df['nouns'] = temp_df['text'].apply(extract_nouns)

# Join the extracted nouns with spaces in temp_df
temp_df['joined_nouns'] = temp_df['nouns'].apply(lambda x: " ".join(x))

# Overwrite the 'text' column in the original DataFrame (df) with the joined nouns
df['text'] = temp_df['joined_nouns']

df.to_csv(output_file_path, index=False)

In [34]:
import pandas as pd

# Load the data into a DataFrame
file_path = "../data/train_cleaning_v4.csv"
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

# Replace all numbers in the 'text' column with '2'
df['text'] = df['text'].replace(to_replace=r'\d', value='2', regex=True)

# Display the DataFrame after the transformation
df.head()

# Save the modified DataFrame to a new CSV file if needed
df.to_csv("../data/train_cleaning_v5.csv", index=False)

In [56]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-wrf7ol_7
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-wrf7ol_7
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
